# 6.2. Convoluciones para imágenes

Ahora que entendemos cómo funcionan las capas convolucionales en teoría, estamos listos para ver cómo funcionan en la práctica. Sobre la base de nuestra motivación de las redes neuronales convolucionales como arquitecturas eficientes para explorar la estructura en los datos de imagen, nos quedamos con las imágenes como nuestro ejemplo de ejecución.

## 6.2.1. La operación de correlación cruzada
Recuerde que estrictamente hablando, las capas convolucionales son un nombre inapropiado, ya que las operaciones que expresan se describen con mayor precisión como correlaciones cruzadas. Con base en nuestras descripciones de capas convolucionales en la Sección 6.1, en dicha capa, un tensor de entrada y un tensor kernel se combinan para producir un tensor de salida a través de una operación de correlación cruzada.

Ignoremos los canales por ahora y veamos cómo funciona esto con datos bidimensionales y representaciones ocultas. En la figura 6.2.1, la entrada es un tensor bidimensional con una altura de 3 y un ancho de 3. Marcamos la forma del tensor como 3×3 o (3, 3). La altura y el ancho del kernel son ambos 2. La forma de la ventana del kernel (o ventana de convolución) está dada por la altura y el ancho del kernel (aquí es 2×2).

../_images/correlación.svg
Fig. 6.2.1 Operación de correlación cruzada bidimensional. Las partes sombreadas son el primer elemento de salida, así como los elementos de tensor kernel y de entrada utilizados para el cálculo de salida: 0×0+1×1+3×2+4×3=19 .

En la operación de correlación cruzada bidimensional, comenzamos con la ventana de convolución ubicada en la esquina superior izquierda del tensor de entrada y la deslizamos a lo largo del tensor de entrada, tanto de izquierda a derecha como de arriba hacia abajo. Cuando la ventana de convolución se desliza a una determinada posición, el subtensor de entrada contenido en esa ventana y el tensor kernel se multiplican por elementos y el tensor resultante se suma dando un único valor escalar. Este resultado da el valor del tensor de salida en la ubicación correspondiente. Aquí, el tensor de salida tiene una altura de 2 y un ancho de 2 y los cuatro elementos se derivan de la operación de correlación cruzada bidimensional:

(6.2.1)
0×0+1×1+3×2+4×3=19,1×0+2×1+4×2+5×3=25,3×0+4×1+6×2+7× 3=37,4×0+5×1+7×2+8×3=43.
 
Tenga en cuenta que a lo largo de cada eje, el tamaño de salida es ligeramente más pequeño que el tamaño de entrada. Debido a que el kernel tiene una anchura y una altura superiores a uno, solo podemos calcular correctamente la correlación cruzada para las ubicaciones en las que el kernel se ajusta completamente a la imagen. El tamaño de salida viene dado por el tamaño de entrada nh×nw menos el tamaño del kernel de convolución. kh×kw vía

(6.2.2)
(nh−kh+1)×(nw−kw+1).
 
Este es el caso ya que necesitamos suficiente espacio para "desplazar" el kernel de convolución a lo largo de la imagen. Más adelante veremos cómo mantener el tamaño sin cambios rellenando la imagen con ceros alrededor de su límite para que haya suficiente espacio para cambiar el kernel. A continuación, implementamos este proceso en la función corr2d, que acepta un tensor de entrada X y un tensor kernel K y devuelve un tensor de salida Y.

In [2]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
#use PDL qw(pdl);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;
use constant is_train =>'False';

In [31]:
sub corr2d{
    my ($X, $K) = @_;
    my ($h, $w) = ($K->[0], $K->[1]);
    my $Y = [];
    for (my $i = 0; $i < scalar(@$X) - $h + 1; $i++) {
        for (my $j = 0; $j < scalar(@{$X->[0]}) - $w + 1; $j++) {
        my $sum = 0;
            for (my $k = 0; $k < $h; $k++) {
            for (my $l = 0; $l < $w; $l++) {
            $sum += $X->[$i+$k][$j+$l] * $K->[$k][$l];
            }
        }
    push @{$Y->[$i]}, $sum;
            }
        }
return $Y;
}

Warning: Subroutine corr2d redefined at reply input line 1.


Podemos construir el tensor de entrada X y el tensor kernel K de la figura 6.2.1 para validar la salida de la implementación anterior de la operación de correlación cruzada bidimensional.

In [32]:
my $X = [
  [0.0, 1.0, 2.0],
  [3.0, 4.0, 5.0],
  [6.0, 7.0, 8.0]
];

my $K = [
  [0.0, 1.0],
  [2.0, 3.0]
];


print corr2d($X, $K);


ARRAY(0xb211908)

1

## 6.2.2. Capas convolucionales
Una capa convolucional correlaciona de forma cruzada la entrada y el kernel y agrega un sesgo escalar para producir una salida. Los dos parámetros de una capa convolucional son el núcleo y el sesgo escalar. Cuando entrenamos modelos basados ​​en capas convolucionales, normalmente inicializamos los núcleos aleatoriamente, tal como lo haríamos con una capa completamente conectada.

Ahora estamos listos para implementar una capa convolucional bidimensional basada en la función corr2d definida anteriormente. En la función constructora __init__, declaramos el peso y el sesgo como los dos parámetros del modelo. La función de propagación directa llama a la función corr2d y agrega el sesgo.

In [ ]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))

    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

In  h×w  convolution or a  h×w  convolution kernel, the height and width of the convolution kernel are  h  and  w , respectively. We also refer to a convolutional layer with a  h×w  convolution kernel simply as a  h×w  convolutional layer.

## 6.2.3. Detección de bordes de objetos en imágenes
Tomemos un momento para analizar una aplicación simple de una capa convolucional: detectar el borde de un objeto en una imagen encontrando la ubicación del cambio de píxel. Primero, construimos una “imagen” de 6×8 píxeles. Las cuatro columnas del medio son negras (0) y el resto son blancas (1).

In [ ]:
use PDL;

my $X = ones(6, 8);
$X->slice("2:6") .= 0;
print $X;


A continuación, construimos un kernel K con una altura de 1 y un ancho de 2. Cuando realizamos la operación de correlación cruzada con la entrada, si los elementos adyacentes horizontalmente son iguales, la salida es 0. De lo contrario, la salida no es -cero.

In [ ]:
use PDL;

my $K = pdl([1.0, -1.0]);

Estamos listos para realizar la operación de correlación cruzada con los argumentos X (nuestra entrada) y K (nuestro kernel). Como puede ver, detectamos 1 para el borde de blanco a negro y -1 para el borde de negro a blanco. Todas las demás salidas toman valor 0.

In [ ]:
my $Y = corr2d($X, $K);
print $Y;

Ahora podemos aplicar el kernel a la imagen transpuesta. Como era de esperar, desaparece. El kernel K solo detecta bordes verticales.

In [ ]:
print corr2d($X->$T, $K);

## 6.2.4. Aprendiendo un núcleo
Diseñar un detector de bordes por diferencias finitas [1, -1] es genial si sabemos que esto es precisamente lo que estamos buscando. Sin embargo, a medida que observamos núcleos más grandes y consideramos capas sucesivas de convoluciones, puede ser imposible especificar con precisión qué debería hacer cada filtro manualmente.

Ahora veamos si podemos aprender el núcleo que generó Y a partir de X observando solo los pares de entrada-salida. Primero construimos una capa convolucional e inicializamos su kernel como un tensor aleatorio. A continuación, en cada iteración, usaremos el error cuadrático para comparar Y con la salida de la capa convolucional. Luego podemos calcular el gradiente para actualizar el kernel. En aras de la simplicidad, a continuación usamos la clase incorporada para capas convolucionales bidimensionales e ignoramos el sesgo.

In [ ]:
use PDL;

# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
my $conv2d = zeros(1, 1, 1, 2);

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
my $X = ones(6, 8)->reshape(1, 1, 6, 8);
$X->slice(":,2:5") .= 0;
my $Y = ones(6, 7)->reshape(1, 1, 6, 7);
my $lr = 0.03;  # Learning rate

for my $i (0 .. 9) {
  my $Y_hat = conv2d_forward($X, $conv2d);
  my $l = ($Y_hat - $Y) ** 2;
  # Update the kernel
  $conv2d -= $lr * $l->sumover * $X;
  if ($i + 1) % 2 == 0) {
    print "batch ", $i + 1, ", loss ", $l->sum, "\n";
  }
}

sub conv2d_forward {
  my ($X, $conv2d) = @_;
  my $output_width = $X->shape->at(3) - $conv2d->shape->at(3) + 1;
  my $output_height = $X->shape->at(2);
  my $Y_hat = zeros($X->shape->at(0), $conv2d->shape->at(1),
                   $output_height, $output_width);
  for my $i (0 .. $output_height - 1) {
    for my $j (0 .. $output_width - 1) {
      my $start = $j;
      my $end = $j + $conv2d->shape->at(3);
      $Y_hat->slice(":,:,$i,$j") .= sum($X->slice(":,:,$i,$start:$end")
                                       * $conv2d);
    }
  }
  return $Y_hat;
}


Tenga en cuenta que el error se redujo a un valor pequeño después de 10 iteraciones. Ahora vamos a echar un vistazo al tensor kernel que aprendimos.

In [ ]:
$conv2d->weight->data->reshape([1, 2]);

De hecho, el tensor kernel aprendido está notablemente cerca del tensor kernel K que definimos anteriormente.

## 6.2.5. Correlación cruzada y convolución
Recuerde nuestra observación de la Sección 6.1 de la correspondencia entre las operaciones de correlación cruzada y convolución. Aquí continuemos considerando capas convolucionales bidimensionales. ¿Qué pasa si tales capas realizan operaciones de convolución estrictas como se define en (6.1.6) en lugar de correlaciones cruzadas? Para obtener el resultado de la operación de convolución estricta, solo necesitamos voltear el tensor kernel bidimensional tanto horizontal como verticalmente, y luego realizar la operación de correlación cruzada con el tensor de entrada.

Cabe señalar que, dado que los núcleos se aprenden de los datos en el aprendizaje profundo, las salidas de las capas convolucionales no se ven afectadas, independientemente de que dichas capas realicen las operaciones de convolución estricta o las operaciones de correlación cruzada.

Para ilustrar esto, suponga que una capa convolucional realiza una correlación cruzada y aprende el núcleo de la figura 6.2.1, que aquí se denota como la matriz K. Suponiendo que otras condiciones permanecen sin cambios, cuando esta capa realiza una convolución estricta, el kernel aprendido K 'será el mismo que K después de que K' se invierta tanto horizontal como verticalmente. Es decir, cuando la capa convolucional realiza una convolución estricta para la entrada de la Fig. 6.2.1 y K′, se obtendrá la misma salida de la Fig. 6.2.1 (correlación cruzada de la entrada y K).

De acuerdo con la terminología estándar de la literatura de aprendizaje profundo, continuaremos refiriéndonos a la operación de correlación cruzada como una convolución aunque, estrictamente hablando, es ligeramente diferente. Además, usamos el término elemento para referirnos a una entrada (o componente) de cualquier tensor que represente una representación de capa o un núcleo de convolución.

## 6.2.6. Mapa de características y campo receptivo
Como se describe en la Sección 6.1.4.1, la salida de la capa convolucional en la Fig. 6.2.1 a veces se denomina mapa de características, ya que puede considerarse como las representaciones (características) aprendidas en las dimensiones espaciales (por ejemplo, ancho y alto) para el capa posterior. En las CNN, para cualquier elemento x de alguna capa, su campo receptivo se refiere a todos los elementos (de todas las capas anteriores) que pueden afectar el cálculo de x durante la propagación directa. Tenga en cuenta que el campo receptivo puede ser más grande que el tamaño real de la entrada.

Sigamos usando la figura 6.2.1 para explicar el campo receptivo. Dado el kernel de convolución 2×2, el campo receptivo del elemento de salida sombreado (de valor 19) son los cuatro elementos en la parte sombreada de la entrada. Ahora denotemos la salida 2×2 como Y y consideremos una CNN más profunda con una capa convolucional adicional de 2×2 que toma Y como su entrada, generando un solo elemento z. En este caso, el campo receptivo de z en Y incluye los cuatro elementos de Y , mientras que el campo receptivo de la entrada incluye los nueve elementos de entrada. Por lo tanto, cuando cualquier elemento en un mapa de características necesita un campo receptivo más grande para detectar características de entrada en un área más amplia, podemos construir una red más profunda.